#Student Name: Martin Power
#Student ID: 9939245

## Etivity4 : Vectors, Matrics and Tensors

Building on my peers code found [here](https://gitlab.com/CE4021/Group2/week3/tree/e28267c1071e79b67d522f9f6d89659c6f650a14)

I have done the following:
 - converted functions to a matrix class
 - implemented addition/subtraction/scalar multiply and matrix multiply as overrides of the arithmetic operators: + - * @ 
 - I have provided additional functions to calculate:
    - the determinant (NxN matrices)
    - the inverse (only 2x2 matrices)
    - the cross product of two suitable vectors (only 3d vectors)
 - confirmed functionality with additional unit tests
 


In [13]:
class Matrix:

    def __init__(self, rows) -> None:
        self.rows = rows
        self.shape = self.__matrix_size()

    def __matrix_size(self):
        # Calculate the size of the tensor as a 2D tuple
        """Function that accepts a matrix in the form of a list of lists and returns
           a tuple indicating matrix size
           Returns (Number of Rows, Number of Columns)
        """
        nr = len(self.rows)
        nc = len(self.rows[0])
        return (nr, nc)

    def __matrix_init(self, size):
        """Function to initialize a matrix. Size received as a tuple of the form
           (Number of Rows, Number of Columns)
           Returns matrix in specified size as a list of lists
        """
        # Code taken from https://www.programiz.com/python-programming/matrix
        nr, nc = size
        n = [0] * nr
        for i in range(nr):
            n[i] = [0] * nc
        return n

    def __add__(self, other_matrix):
        return self.__matrix_add_sub(other_matrix)

    def __sub__(self, other_matrix):
        return self.__matrix_add_sub(other_matrix, op="sub")

    def __matrix_add_sub(self, m, op="add"):
        # Sum/subtract the tensor with another tensor of suitable size
        """Function to add or substract two matrices. Operation is specified using
        'op' string and defaults to addition
        """
        (nr, nc) = self.shape

        # Matrix Size Mismatch. Addition/Subtraction Not Possible
        if (m.shape) != (nr, nc):
            return "ERROR"
        else:
            result = self.__matrix_init((nr, nc))

        for i in range(nr):
            for j in range(nc):
                if op == 'sub':
                    result[i][j] = self.rows[i][j] - m.rows[i][j]
                else:
                    result[i][j] = self.rows[i][j] + m.rows[i][j]
        rows = [tuple(i) for i in result]
        return Matrix(tuple(rows))

    def __mul__(self, m):
        # Multiply the tensor with a scalar
        """Function to multiply a matrix by a scalar.
        """
        
        nr, nc = self.shape
        result = self.__matrix_init(self.shape)
        for i in range(nr):
            for j in range(nc):
                result[i][j] = self.rows[i][j] * m
        rows = (tuple(i) for i in result)
        return Matrix(tuple(rows))


    def __matmul__(self, m):
       # Multiply the tensor with a tensor of suitable size or tensor by a scalar
        """Function to multiply two matrices. If matrix dimensions prevent
        multiplication, ERROR is returned
        """
       
        (nr_n, nc_n) = self.shape
        (nr_m, nc_m) = m.shape

        # Number of Rows in Matrix N must equal Number of Columns in Matrix M
        if (nc_n != nr_m):
            return "ERROR"
        else:
            result = self.__matrix_init((nr_n, nc_m))

        for i in range(nr_n):
            for j in range(nc_m):
                for k in range(nc_n):  # Could also have used nr_m as nc_n==nr_m
                    result[i][j] += self.rows[i][k] * m.rows[k][j]
        rows = (tuple(i) for i in result)
        return Matrix(tuple(rows))

    def __str__(self):
        return str(self.rows)

    def __eq__(self, other_matrix):
        return self.rows == other_matrix.rows

    def determinant(self):
        # Calculate the determinant of the tensor, but only for square tensors (matrix) I have used a recursive call
        # which can calculate the determinant of any square matrix by splitting the problem down
        (nr, nc) = self.shape
        if nr != nc:
            raise ValueError("implementation only covers square matrices")
        elif self.shape == (2,2):
            return (self.rows[0][0] * self.rows[1][1]) - (self.rows[1][0]) * self.rows[0][1]
        else:
            determinant = 0
            for i, column in enumerate(self.rows[0]):
                pivot = column
                pivot_matrix = self.__get_matrix_excluding_row_and_column(0, i)
                determinant += ((-1)**(2+ i)) * pivot * pivot_matrix.determinant()
            return determinant

    def __get_matrix_excluding_row_and_column(self, row_index, column_index):
        nr, nc = self.shape
        rows = []
        for i in range(nr):
            if row_index != i:
                row = []
                for j in range(nc):
                    if column_index != j:
                        row.append(self.rows[i][j])
                rows.append(tuple(row))
        return Matrix(tuple(rows))

    def inverse(self):
        # Calculate the inverse of the tensor, but only for tensors of size 2x2
        determinant = self.determinant()
        if determinant == 0:
            return None
        a = self.rows[0][0] / determinant
        b = self.rows[0][1] / determinant
        c = self.rows[1][0] / determinant
        d = self.rows[1][1] / determinant
        return Matrix(((d, -b),(-c,a)))


    def cross_product(self, other_vector):
        # Calculate the cross product of the tensor with another tensor of suitable size
        #This implementation only works with vectors with rank 3, other rank vectors other than 7 do not have a cross product

        (nr_n, nc_n) = self.shape
        (nr_m, nc_m) = other_vector.shape

        if nr_n != 3 or nr_m != 3 or nc_n != 1 or nc_m != 1:
            raise ValueError("can only calculate cross product for 3d vectors")

        rows = []
        for i in range(3):
            column = (0, self.rows[i][0], other_vector.rows[i][0])
            rows.append(column)
        cross_product_matrix = Matrix(tuple(rows))

        iHat = Matrix(((1,),(0,),(0,)))
        jHat = Matrix(((0,),(1,),(0,)))
        kHat = Matrix(((0,),(0,),(1,)))

        a = iHat * cross_product_matrix.__get_matrix_excluding_row_and_column(0, 0).determinant()
        b = jHat * -cross_product_matrix.__get_matrix_excluding_row_and_column(1, 0).determinant()
        c = kHat * cross_product_matrix.__get_matrix_excluding_row_and_column(2, 0).determinant()

        return a + b + c

In [14]:
##################################################
# Unit Test
##################################################
import unittest

a = Matrix(((-2, 7), (9, 4)))  # A: 2x2 Matrix
b = Matrix(((4, 8), (-6, 1))) # B: 2x2 Matrix
c = Matrix(((8, 2),))  # C: 1x2 Row Vector
d = Matrix(((-3,), (4,)))  # D: 2x1 Column Vector


class TestEtivity3(unittest.TestCase):
    # Test Matrix Addition and Subtraction
    def test_001(self):
        expected_result = Matrix(((2, 15), (3, 5)))  # A+B
        self.assertEqual(a + b, expected_result, "A+B Mismatch")

        expected_result = Matrix(((-6, -1), (15, 3)))  # A-B
        self.assertEqual(a - b, expected_result, "A-B Mismatch")


# Test Matrix Multiplication
    def test_002(self):
        expected_result = Matrix(((-50, -9), (12, 76)))  # A*B
        self.assertEqual(a @ b, expected_result, "A*B Mismatch")

        expected_result = Matrix(((64, 60), (21, -38))) # B*A
        self.assertEqual(b @ a, expected_result, "B*A Mismatch")

        expected_result = Matrix(((2, 64),))  # C*A
        self.assertEqual(c @ a, expected_result, "C*A Mismatch")

        expected_result = Matrix(((34,), (-11,)))  # A*D
        self.assertEqual(a @ d, expected_result, "A*D Mismatch")

        expected_result = Matrix(((-16,),))  # C*D
        self.assertEqual(c @ d, expected_result, "C*D Mismatch")

        expected_result = Matrix(((-24, -6), (32, 8)))  # D*C
        self.assertEqual(d @ c, expected_result, "D*C Mismatch")

        expected_result = "ERROR"  # A*C
        self.assertEqual(a @ c, expected_result, "A*C Mismatch")
        
        expected_result = Matrix(((-4, 14), (18, 8)))  # A*C
        self.assertEqual(a * 2, expected_result, "A*C Mismatch")

# Test Matrix determinant, inverse and cross product
    def test_003(self):
        matrix = Matrix(((5,7),(2,3)))
        expected_result = 1
        self.assertEqual(matrix.determinant(), expected_result)

        matrix = Matrix(((5,2),(-7,-3)))
        expected_result = Matrix(((3,2),(-7,-5)))
        self.assertEqual(matrix.inverse(), expected_result)

        # expected_result = Matrix((-44, 33))
        # self.assertEqual(a.cross_product(b), expected_result)

        matrix = Matrix(((2,-3,1),(2,0,-1),(1,4,5)))
        expected_result = 49
        self.assertEqual(matrix.determinant(), expected_result)

        vector_a = Matrix(((1,),(2,),(3,)))
        vector_b = Matrix(((3,),(2,),(1,)))
        expected_result = Matrix(((-4,),(8,),(-4,)))
        actual_result = vector_a.cross_product(vector_b)
        self.assertEqual(actual_result, expected_result)

        vector_a = Matrix(((1,),(2,),(3,)))
        expected_result = Matrix(((0,),(0,),(0,)))
        actual_result = vector_a.cross_product(vector_a)
        self.assertEqual(actual_result, expected_result)

        vector_a = Matrix(((2,),(3,),(4,)))
        vector_b = Matrix(((5,),(6,),(7,)))
        expected_result = Matrix(((-3,),(6,),(-3,)))
        actual_result = vector_a.cross_product(vector_b)
        self.assertEqual(actual_result, expected_result)





if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


.

.

.


----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
